In [1]:
import pandas as pd
import fnmatch
import dask.dataframe as dd
from intake.source.utils import reverse_format
import os
import re
import subprocess
from tqdm.auto import tqdm
from pathlib import Path
import shutil
import numpy as np

## Create text file containing all files available

In [2]:
def get_file_list(persist_path):
    root = Path("/glade/collections/cmip/cmip5/")
    p_path = Path(persist_path)
    p_path.mkdir(exist_ok=True)
    dirs = [x for x in root.iterdir() if x.is_dir()]
    for directory in tqdm(dirs):
        print(directory)
        stem = directory.stem
        f = open(f"{persist_path}/{stem}.txt", "w")
        cmd = ["find", "-L", directory.as_posix(), "-name", "*.nc"]
        p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=f)
        p.wait()
        

In [3]:
persist_path = "./CMIP5_filelist"
#get_file_list(persist_path)


## Extract attributes of a file using information from CMIP5 DRS.



Reference:
- CMIP5 DRS: https://pcmdi.llnl.gov/mips/cmip5/docs/cmip5_data_reference_syntax.pdf?id=27

Directory:
```
  <activity>/
    <product>/
        <institute>/
            <model>/
                <experiment>/
                    <frequency>/
                        <modeling realm>/
                            <MIP table>/
                                <ensemble member>/
                                    <version number>/
                                        <variable name>/
                                            <CMOR filename>.nc
```
                                                
CMOR filename: `<variable name>_<MIP table>_<model>_<experiment>_ <ensemble member>[_<temporal subset>][_<geographical info>].nc`
"""

In [4]:
products = list(Path(persist_path).rglob("*.txt"))
products = [product.stem for product in products]
products

['output1', 'output2', 'output']

In [5]:
df = dd.read_csv(f"{persist_path}/*.txt", header=None).compute()
df.columns = ["path"]
df.head()

,path
0,/glade/collections/cmip/cmip5/output/CCCma/Can...
1,/glade/collections/cmip/cmip5/output/CCCma/Can...
2,/glade/collections/cmip/cmip5/output/CCCma/Can...
3,/glade/collections/cmip/cmip5/output/CCCma/Can...
4,/glade/collections/cmip/cmip5/output/CCCma/Can...


In [6]:
len(df)

927318

In [7]:
def _reverse_filename_format(file_basename, filename_template=None, gridspec_template=None):
    """
    Uses intake's ``reverse_format`` utility to reverse the string method format.
    Given format_string and resolved_string, find arguments
    that would give format_string.format(arguments) == resolved_string
    """
    try:
        return reverse_format(filename_template, file_basename)
    except ValueError:
        try:
            return reverse_format(gridspec_template, file_basename)
        except:
            print(
                f'Failed to parse file: {file_basename} using patterns: {filename_template} and {gridspec_template}'
            )
            return {}
            
def _extract_attr_with_regex(input_str, regex, strip_chars=None):
    pattern = re.compile(regex, re.IGNORECASE)
    match = re.findall(pattern, input_str)
    if match:
        match = max(match, key=len)
        if strip_chars:
            match = match.strip(strip_chars)

        else:
            match = match.strip()

        return match

    else:
        return None
    

exclude_patterns = ['*/files/*', '*/latest/*']
def _filter_func(path):
    return not any(
        fnmatch.fnmatch(path, pat=exclude_pattern) for exclude_pattern in exclude_patterns
    )



In [8]:
%%time
files = df.path.tolist()
filelist = list(filter(_filter_func, files))

CPU times: user 3.47 s, sys: 47.9 ms, total: 3.52 s
Wall time: 3.51 s


In [9]:
len(filelist)

629942

In [10]:

def get_attrs(filepath):
    """ Extract attributes of a file using information from CMIP5 DRS.
    Notes
    -----
    Reference:
    - CMIP5 DRS: https://pcmdi.llnl.gov/mips/cmip5/docs/cmip5_data_reference_syntax.pdf?id=27
    """

    fileparts = {}

    freq_regex = r'/3hr/|/6hr/|/day/|/fx/|/mon/|/monClim/|/subhr/|/yr/'
    realm_regex = r'aerosol|atmos|land|landIce|ocean|ocnBgchem|seaIce'
    version_regex = r'v\d{4}\d{2}\d{2}|v\d{1}'

    file_basename = os.path.basename(filepath)
    fileparts['path'] = filepath

    filename_template = (
        '{variable}_{mip_table}_{model}_{experiment}_{ensemble_member}_{temporal_subset}.nc'
    )
    gridspec_template = '{variable}_{mip_table}_{model}_{experiment}_{ensemble_member}.nc'
    f = _reverse_filename_format(
        file_basename, filename_template=filename_template, gridspec_template=gridspec_template
    )
    fileparts.update(f)

    frequency = _extract_attr_with_regex(
        filepath, regex=freq_regex, strip_chars='/'
    )
    realm = _extract_attr_with_regex(filepath, regex=realm_regex)
    version = _extract_attr_with_regex(filepath, regex=version_regex) or 'v0'
    fileparts['frequency'] = frequency
    fileparts['modeling_realm'] = realm
    fileparts['version'] = version
    try:
        part1, part2 = os.path.dirname(filepath).split(fileparts['experiment'])
        part1 = part1.strip("/").split("/")
        fileparts['institute'] = part1[-2]
        fileparts['product_id'] = part1[-3]
    except Exception:
        print(fileparts)
    

    return fileparts

In [11]:
get_attrs(filelist[0])

{'path': '/glade/collections/cmip/cmip5/output/CCCma/CanCM4/historical/mon/atmos/r10i1p1/v20130331/hfls/hfls_Amon_CanCM4_historical_r10i1p1_196101-200512.nc',
 'variable': 'hfls',
 'mip_table': 'Amon',
 'model': 'CanCM4',
 'experiment': 'historical',
 'ensemble_member': 'r10i1p1',
 'temporal_subset': '196101-200512',
 'frequency': 'mon',
 'modeling_realm': 'atmos',
 'version': 'v20130331',
 'institute': 'CCCma',
 'product_id': 'output'}

In [12]:
%%time
entries = list(map(get_attrs, filelist))

CPU times: user 30.3 s, sys: 1.06 s, total: 31.4 s
Wall time: 31.4 s


In [13]:
entries[10]

{'path': '/glade/collections/cmip/cmip5/output/CCCma/CanCM4/historical/mon/atmos/r9i1p1/v20130331/tas/tas_Amon_CanCM4_historical_r9i1p1_196101-200512.nc',
 'variable': 'tas',
 'mip_table': 'Amon',
 'model': 'CanCM4',
 'experiment': 'historical',
 'ensemble_member': 'r9i1p1',
 'temporal_subset': '196101-200512',
 'frequency': 'mon',
 'modeling_realm': 'atmos',
 'version': 'v20130331',
 'institute': 'CCCma',
 'product_id': 'output'}

In [14]:
len(entries)

629942

In [15]:
df = pd.DataFrame(entries)
df = df.drop_duplicates(subset=['path'], keep='last').reset_index(drop=True)
df.head()

,path,variable,mip_table,model,experiment,ensemble_member,temporal_subset,frequency,modeling_realm,version,institute,product_id
0,/glade/collections/cmip/cmip5/output/CCCma/Can...,hfls,Amon,CanCM4,historical,r10i1p1,196101-200512,mon,atmos,v20130331,CCCma,output
1,/glade/collections/cmip/cmip5/output/CCCma/Can...,tas,Amon,CanCM4,historical,r10i1p1,196101-200512,mon,atmos,v20130331,CCCma,output
2,/glade/collections/cmip/cmip5/output/CCCma/Can...,pr,Amon,CanCM4,historical,r10i1p1,196101-200512,mon,atmos,v20130331,CCCma,output
3,/glade/collections/cmip/cmip5/output/CCCma/Can...,hfls,Amon,CanCM4,historical,r1i1p1,196101-200512,mon,atmos,v20130331,CCCma,output
4,/glade/collections/cmip/cmip5/output/CCCma/Can...,tas,Amon,CanCM4,historical,r1i1p1,196101-200512,mon,atmos,v20130331,CCCma,output


In [16]:
len(df)

629942

In [17]:
# Some entries are invalid
invalids = df[~df.product_id.isin(products)]
invalids
#df = df[df.activity_id.isin(activity_ids)]

,path,variable,mip_table,model,experiment,ensemble_member,temporal_subset,frequency,modeling_realm,version,institute,product_id


In [18]:
df = df[df.product_id.isin(products)]
len(df)

629942

In [19]:
df.ensemble_member.unique()

array(['r10i1p1', 'r1i1p1', 'r6i1p1', 'r9i1p1', 'r3i1p1', 'r5i1p1',
       'r8i1p1', 'r2i1p1', 'r4i1p1', 'r7i1p1', 'r1i1p2', 'r1i1p4',
       'r1i1p3', 'r1i2p1', 'r3i2p1', 'r2i2p1', 'r0i0p0', 'r4i2p1',
       'r7i2p1', 'r9i2p1', 'r6i2p1', 'r8i2p1', 'r5i2p1', 'r10i2p1',
       'r1i2p2', 'r4i1p14', 'r6i1p12', 'r4i1p10', 'r1i1p15', 'r6i1p14',
       'r6i1p11', 'r1i1p16', 'r4i1p15', 'r4i1p11', 'r1i1p17', 'r1i1p11',
       'r6i1p10', 'r6i1p17', 'r2i1p11', 'r1i1p12', 'r4i1p16', 'r1i1p10',
       'r4i1p12', 'r6i1p16', 'r1i1p13', 'r6i1p13', 'r1i1p14', 'r6i1p15',
       'r4i1p17', 'r3i1p13', 'r1i1p8', 'r1i1p7', 'r1i1p6', 'r1i1p5',
       'r6i1p2', 'r6i1p3', 'r3i1p3', 'r4i1p3', 'r5i1p3', 'r5i1p2',
       'r4i1p2', 'r3i1p2', 'r2i1p2', 'r2i1p3', 'r2i1p302', 'r1i1p109',
       'r1i1p105', 'r1i1p101', 'r1i1p312', 'r4i1p114', 'r3i1p313',
       'r3i1p106', 'r2i1p114', 'r3i1p309', 'r2i1p109', 'r5i1p303',
       'r5i1p311', 'r1i1p104', 'r5i1p102', 'r3i1p101', 'r1i1p311',
       'r1i1p3031', 'r5i1p313',

## Pick the latest versions only

In [20]:
grpby = list(set(df.columns.tolist()) - {'path', 'version'})
groups = df.groupby(grpby)

In [21]:
%%time
idx_to_remove = []
for _, group in groups:
    if group.version.nunique() > 1:
        idx_to_remove.extend(group.sort_values(by=['version'], ascending=False).index[1:].values.tolist())

CPU times: user 2min 23s, sys: 3.86 s, total: 2min 26s
Wall time: 2min 26s


In [22]:
len(idx_to_remove)

303368

In [23]:
len(df)

629942

In [24]:
df1 = df.copy()
df = df.drop(index=idx_to_remove)
len(df)

326574

In [25]:
df.head()

,path,variable,mip_table,model,experiment,ensemble_member,temporal_subset,frequency,modeling_realm,version,institute,product_id
0,/glade/collections/cmip/cmip5/output/CCCma/Can...,hfls,Amon,CanCM4,historical,r10i1p1,196101-200512,mon,atmos,v20130331,CCCma,output
1,/glade/collections/cmip/cmip5/output/CCCma/Can...,tas,Amon,CanCM4,historical,r10i1p1,196101-200512,mon,atmos,v20130331,CCCma,output
2,/glade/collections/cmip/cmip5/output/CCCma/Can...,pr,Amon,CanCM4,historical,r10i1p1,196101-200512,mon,atmos,v20130331,CCCma,output
3,/glade/collections/cmip/cmip5/output/CCCma/Can...,hfls,Amon,CanCM4,historical,r1i1p1,196101-200512,mon,atmos,v20130331,CCCma,output
4,/glade/collections/cmip/cmip5/output/CCCma/Can...,tas,Amon,CanCM4,historical,r1i1p1,196101-200512,mon,atmos,v20130331,CCCma,output


In [26]:
df.columns.shape

(12,)

In [27]:
# Re-arange columns
columns = ["product_id", "institute", "model", "experiment", "frequency", 
           "modeling_realm", "mip_table", "ensemble_member", "variable", 
           "temporal_subset", "version", "path"]
df = df[columns]
df.head()

,product_id,institute,model,experiment,frequency,modeling_realm,mip_table,ensemble_member,variable,temporal_subset,version,path
0,output,CCCma,CanCM4,historical,mon,atmos,Amon,r10i1p1,hfls,196101-200512,v20130331,/glade/collections/cmip/cmip5/output/CCCma/Can...
1,output,CCCma,CanCM4,historical,mon,atmos,Amon,r10i1p1,tas,196101-200512,v20130331,/glade/collections/cmip/cmip5/output/CCCma/Can...
2,output,CCCma,CanCM4,historical,mon,atmos,Amon,r10i1p1,pr,196101-200512,v20130331,/glade/collections/cmip/cmip5/output/CCCma/Can...
3,output,CCCma,CanCM4,historical,mon,atmos,Amon,r1i1p1,hfls,196101-200512,v20130331,/glade/collections/cmip/cmip5/output/CCCma/Can...
4,output,CCCma,CanCM4,historical,mon,atmos,Amon,r1i1p1,tas,196101-200512,v20130331,/glade/collections/cmip/cmip5/output/CCCma/Can...


In [28]:
df.to_csv("../catalogs/glade-cmip5.csv.gz", compression="gzip", index=False)